In [1]:
import pandas as pd
from transformers import AutoTokenizer
import tensorflow as tf
import numpy as np
from app import pre_process  # Importe ta fonction
from sklearn.metrics import classification_report


c:\Users\invité\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-05-14 19:30:33.812 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-14 19:30:33.817 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-14 19:30:38.968 
  command:

    streamlit run C:\Users\invité\AppData\Roaming\Python\Python312\site-packages\ipykernel_launcher.py [ARGUMENTS]
2025-05-14 19:30:38.974 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-14 19:30:38.974 Thread 'MainThread': missing ScriptRu

In [2]:
# Charger le modèle et le tokenizer
tokenizer = AutoTokenizer.from_pretrained("distilbert/distilbert-base-uncased")
saved_model = tf.saved_model.load('tweet_sentiment_analysis/model_distilBERT')
inference_func = saved_model.signatures['serving_default']

In [3]:
# Charger le modèle et le tokenizer

def predict(text):
    cleaned_text = pre_process(text)
    tokens = tokenizer(cleaned_text, return_tensors="tf", max_length=128, truncation=True, 
                      padding="max_length", return_attention_mask=True, return_token_type_ids=False)
    outputs = inference_func(input_ids=tokens['input_ids'], attention_mask=tokens['attention_mask'])
    logits = outputs['logits']
    pred = np.argmax(logits, axis=1)[0]
    return pred  # 0, 1 ou 2

In [4]:
# Charger les données
df = pd.read_csv("test.csv") 
df.head()

,textID,text,sentiment
0,f87dea47db,Last session of the day http://twitpic.com/67ezh,neutral
1,96d74cb729,Shanghai is also really exciting (precisely -...,positive
2,eee518ae67,"Recession hit Veronique Branquinho, she has to...",negative
3,01082688c6,happy bday!,positive
4,33987a8ee5,http://twitpic.com/4w75p - I like it!!,positive


In [5]:
# Map des labels texte en index
df_clean = df.dropna(subset=['sentiment'])
label_mapping = {'negative': 0, 'neutral': 1, 'positive': 2}
true_labels = df_clean['sentiment'].map(label_mapping).tolist()

preds_en = [predict(text) for text in df_clean['text']]


In [6]:
print("== Évaluation en Anglais ==")
print(classification_report(true_labels, preds_en, target_names=["Négatif", "Neutre", "Positif"]))

== Évaluation en Anglais ==
              precision    recall  f1-score   support

     Négatif       0.77      0.80      0.78      1001
      Neutre       0.76      0.74      0.75      1430
     Positif       0.83      0.83      0.83      1103

    accuracy                           0.78      3534
   macro avg       0.79      0.79      0.79      3534
weighted avg       0.78      0.78      0.78      3534



In [7]:
# Charger les données
df_french = pd.read_csv("test_traduit_francais.csv") 
df_french.head()

,textID,text,sentiment,text_fr
0,f87dea47db,Last session of the day http://twitpic.com/67ezh,neutral,Dernière session du jour http://twitpic.com/67ezh
1,96d74cb729,Shanghai is also really exciting (precisely -...,positive,Shanghai est aussi très excitante (précisément...
2,eee518ae67,"Recession hit Veronique Branquinho, she has to...",negative,"La récession a frappé Veronique Branquinho, el..."
3,01082688c6,happy bday!,positive,Joyeux bday !
4,33987a8ee5,http://twitpic.com/4w75p - I like it!!,positive,http://twitpic.com/4w75p - J'aime bien!!


In [10]:
# Map des labels texte en index
df_clean_french = df_french.dropna(subset=['sentiment'])
label_mapping_french = {'negative': 0, 'neutral': 1, 'positive': 2}
true_labels_french = df_clean_french['sentiment'].map(label_mapping_french).tolist()

preds_fr = [predict(text) for text in df_clean_french['text_fr']]


In [11]:
# Évaluer en français
print("== Évaluation en Français ==")
print(classification_report(true_labels_french, preds_fr, target_names=["Négatif", "Neutre", "Positif"]))

== Évaluation en Français ==
              precision    recall  f1-score   support

     Négatif       0.66      0.02      0.04      1001
      Neutre       0.42      0.96      0.58      1430
     Positif       0.68      0.12      0.20      1103

    accuracy                           0.43      3534
   macro avg       0.58      0.37      0.28      3534
weighted avg       0.56      0.43      0.31      3534

